# Open AI Playground

## Setup Open AI

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
OPEN_AI_API_KEY = os.getenv('OPEN_AI_API_KEY')
OPEN_AI_ASST_ID = os.getenv('OPEN_AI_ASST_ID')

client = OpenAI(
    api_key=OPEN_AI_API_KEY
)

## Retrieve Assistant

In [2]:
assistant = client.beta.assistants.retrieve(
    assistant_id=OPEN_AI_ASST_ID
)
assistant

Assistant(id='asst_3RMgZeDtWYs3CPfXnWon2sAm', created_at=1731571626, description=None, instructions='Provide a solution for a 2D maze problem based on a user-provided or generated 2D grid. The grid represents a maze, with cells marked as `0` denoting paths, and `1` denoting obstacles.\nThe maze always starts at (1, 0) and ends at (-2, -1) \n\nExplain every reasoning step before arriving at a solution. The expected solution must accurately reflect the path taken, considering constraints such as obstacles and boundaries. Utilize the provided function or helper methods to solve the maze.\n\nIf the user asks to generate a maze, wait for the generated maze to be submitted by the "generate_maze" function before calling the "solve_maze" function.\n\n# Steps\n\n1. **Input Handling**:\n   - If the user provides a 2D array, use it to solve the maze.\n   - If requested by the user, generate a random 2D grid and proceed to solve it. Use the provided function "generate_maze"\n\n2. **Maze Representa

## Create a Thread
Here we can create a thread, think of it as a new "chat". We can add as many messages we want to the thread. We can see the entire thread history of all user and AI repsonse messages.

In [3]:
thread = client.beta.threads.create()
thread

Thread(id='thread_MH93fHw39ELUSD5LncIWuGea', created_at=1731594694, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

## Add a Message to the Thread
Here we can add our user defined messages to the thread.

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=(
        f"Create a 2D Maze of with grid dimension of 10."
        f"Once the maze is generated, call the solve maze function to solve the maze."
    )
)
message

Message(id='msg_pITWOnJEroJGvhhWzPQhjmS2', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create a 2D Maze of with grid dimension of 10.Once the maze is generated, call the solve maze function to solve the maze.'), type='text')], created_at=1731594694, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MH93fHw39ELUSD5LncIWuGea')

## Create Run and Stream Updates
Here we tell the Assistant to run the messages we added to thread. We also check if the run submitted any tool outputs (ie. Function Calls). If a function call is submitted we check which function was called and run our own function to return the result to Open AI.

In [5]:
import json
from typing_extensions import override
from openai import AssistantEventHandler
from openai.types.beta.threads.run import Run
from maze import solve_maze_bfs, create_maze, highlight_path
from IPython.display import display, Markdown

 
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id
            self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data: Run, run_id):
        tool_outputs = []
            
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "generate_maze":
                generator_inputs: dict = json.loads(tool.function.arguments)
                generator_dim = generator_inputs.get('dimension')
                generated_maze = create_maze(dim=generator_dim)

                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": json.dumps(generated_maze)
                })
            elif tool.function.name == "solve_maze":
                solve_inputs: dict = json.loads(tool.function.arguments)
                solve_maze = solve_inputs.get('grid')
                solved_path = solve_maze_bfs(maze=solve_maze)
                highlighted_maze = highlight_path(solve_maze, solved_path)

                solved_output: dict = {
                    "path_status": "Path Found" if solved_path else "No Path Found",
                    "solution_path": solved_path,
                    "highlighted_maze": highlighted_maze
                }
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": json.dumps(solved_output)
                })
        
        self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            markdown_text = ""
            for text in stream.text_deltas:
                markdown_text += text
                print(text, end="", flush=True)

            display(Markdown(markdown_text))
    
 
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
) as stream:
    stream.until_done()

### Path Status: Path Found

### Original Maze:
```
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1
1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1
1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1
1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1
1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1
1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1
1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1
1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1
1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1
1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1
1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1
1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1
1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1
1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1
1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
```

### Highlighted Maze:
```
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

### Path Status: Path Found

### Original Maze:
```
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1
1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1
1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1
1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1
1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1
1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1
1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1
1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1
1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1
1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1
1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1
1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1
1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1
1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1
1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
```

### Highlighted Maze:
```
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
* * 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
1 * 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
1 * * * * * 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1
1 1 1 1 1 * 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1
1 * * * * * 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1
1 * 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
1 * * * * * 1 * * * 1 0 0 0 1 * * * 1 0 1
1 1 1 1 1 * 1 * 1 * 1 0 1 1 1 * 1 * 1 0 1
1 0 0 0 1 * * * 1 * 1 0 1 * * * 1 * 1 0 1
1 0 1 1 1 1 1 1 1 * 1 1 1 * 1 1 1 * 1 0 1
1 0 0 0 0 0 0 0 1 * * * 1 * * * 1 * * * 1
1 1 1 1 1 1 1 0 1 1 1 * 1 0 1 * 1 1 1 * 1
1 0 0 0 0 0 0 0 0 0 1 * 1 0 1 * * * 1 * 1
1 0 1 0 1 1 1 1 1 1 1 * 1 0 1 1 1 * 1 * 1
1 0 1 0 1 * * * * * * * 1 0 1 * * * 1 * 1
1 0 1 0 1 * 1 1 1 1 1 1 1 1 1 * 1 1 1 * 1
1 0 1 0 1 * 1 0 0 * * * * * 1 * 1 0 1 * 1
1 0 1 1 1 * 1 1 1 * 1 1 1 * 1 * 1 0 1 * 1
1 0 0 0 0 * * * * * 1 0 0 * * * 1 0 0 * *
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
```

### Solution Path:
```
[(1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 5), 
(5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (6, 1), (7, 1), (7, 2), (7, 3), 
(7, 4), (7, 5), (8, 5), (9, 5), (9, 6), (9, 7), (8, 7), (7, 7), (7, 8), 
(7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (11, 10), (11, 11), (12, 11), 
(13, 11), (14, 11), (15, 11), (15, 10), (15, 9), (15, 8), (15, 7), (15, 6), 
(15, 5), (16, 5), (17, 5), (18, 5), (19, 5), (19, 6), (19, 7), (19, 8), 
(19, 9), (18, 9), (17, 9), (17, 10), (17, 11), (17, 12), (17, 13), (18, 13), 
(19, 13), (19, 14), (19, 15), (18, 15), (17, 15), (16, 15), (15, 15), 
(15, 16), (15, 17), (14, 17), (13, 17), (13, 16), (13, 15), (12, 15), 
(11, 15), (11, 14), (11, 13), (10, 13), (9, 13), (9, 14), (9, 15), (8, 15), 
(7, 15), (7, 16), (7, 17), (8, 17), (9, 17), (10, 17), (11, 17), (11, 18), 
(11, 19), (12, 19), (13, 19), (14, 19), (15, 19), (16, 19), (17, 19), 
(18, 19), (19, 19), (19, 20)]
```

### Reasoning Log:
- Starting at (1, 0), moved right to (1, 1).
- Continued downwards through (2, 1) and then to (3, 1).
- Traversed horizontally and vertically, avoiding dead ends or obstacles, moving along the paths that led closest towards the exit at each turn.
- At each junction, prioritized continuing in the current direction unless obstructed, and adjusted the path accordingly.
- Upon reaching the edges of boundaries, carefully maneuvered through available openings to maintain progress towards the goal.
- Eventually, traversed to the final cell at (19, 20), completing the path successfully.